haimtran 12/12/2023

let build chatgpt by andrej kaparthy 

part 1 

In [1]:
# !wget https://raw.g ithubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 

In [3]:
batch_size = 64 # how many independent sequences will we process in parallel 
block_size = 256  # what is the maximum context length for predictions?
max_iters = 5000 
eval_interval = 500 
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200 
# 
n_embed = 384 
n_head = 6 
n_layer = 6 
dropout = 0.2
# 
torch.manual_seed(1337) 

In [5]:
torch.set_default_device(device)

read input text 

In [6]:
with open("input.txt", 'r', encoding='utf-8') as f:
  text = f.read()

In [8]:
len(text)

1115394

In [9]:
chars = sorted(list(set(text)))
vocab_size =len(chars)
print('.'.join(chars))
print(vocab_size)


. .!.$.&.'.,.-...3.:.;.?.A.B.C.D.E.F.G.H.I.J.K.L.M.N.O.P.Q.R.S.T.U.V.W.X.Y.Z.a.b.c.d.e.f.g.h.i.j.k.l.m.n.o.p.q.r.s.t.u.v.w.x.y.z
65


In [10]:
len(chars)

65

build string to index map, encode and decode 

In [11]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

In [12]:
encode = lambda s: [stoi[c] for c in s] # encode take a string and output list of integer

In [13]:
decode = lambda l: ''.join([itos[i] for i in l]) # decode a list of integer to a string 

In [16]:
data = torch.tensor(encode(text))

In [18]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[:n]

In [23]:
def get_batch(split):
  # generate a small batch of data of input x and targets y 
  data = train_data if split == 'train' else val_data 
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([ data[i:i+block_size] for i in ix])
  y = torch.stack([ data[i+1: i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x, y 

In [29]:
class Head(nn.Module):
  
  def __init__(self, head_size) -> None:
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed,  head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)
    
  def forward(self, x):
    B,T,C = x.shape 
    k = self.key(x)   # (B, T, C)
    q = self.query(x) # (B, T, C)
    # compute attention scores ("affinities")
    wei = q @ k.transpose(-2,-1) * C**-0.05 # (B, T, C) @ (B, C, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values 
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
    return out 
  

class FeedForward(nn.Module):

  def __init__(self, n_embed) -> None:
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embed, 4 * n_embed),
      nn.ReLU(),
      nn.Linear(4 * n_embed, n_embed),
      nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size) -> None:
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out 
  

class Block(nn.Module):

  def __init__(self, n_embed, n_head) -> None:
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x 


class BigramLanguageMmodel(nn.Module):

  def __init__(self, vocab_size) -> None:
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_talbe = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed) # final layer norm 
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape 
    
    # idx and targets are both (B, T) tensor of integers 
    tok_emb = self.token_embedding_table(idx) # (B, T, C)
    pos_emb = self.position_embedding_talbe(torch.arange(T, device=device))
    x = tok_emb + pos_emb # (B, T, C)
    x = self.blocks(x) # (B, T, C) 
    x = self.ln_f(x) # (B, T, C)  
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None 
    else: 
      B, T, C = logits.shape 
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      # softmax and loss
      loss = F.cross_entropy(logits, targets)

    return logits, loss 
  

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indicies in the current context 
    for _ in range(max_new_tokens):
      # crop ind to the last block_size tokens 
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last time step 
      logits = logits[:, -1, :] # become (B, C)
      # apply softmax to get probablities 
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [30]:
model = BigramLanguageMmodel(vocab_size)
model = model.to(device)

pytorch optimizer 

In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [32]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out 

In [33]:
# batch_size = 32 
for iter in range(max_iters):

  # every once in a while evaluate the loss on the train and val sets 
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data 
  xb, yb = get_batch("train")
  
  # evaluate the loss 
  logits, loss = model(xb, yb)
  # optimizer step 
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.4150, val loss 4.4153
step 500: train loss 1.8101, val loss 1.8081
step 1000: train loss 1.5178, val loss 1.5173
step 1500: train loss 1.3882, val loss 1.3886
step 2000: train loss 1.3077, val loss 1.3067
step 2500: train loss 1.2566, val loss 1.2569
step 3000: train loss 1.2125, val loss 1.2117
step 3500: train loss 1.1730, val loss 1.1734
step 4000: train loss 1.1388, val loss 1.1392
step 4500: train loss 1.1071, val loss 1.1066


In [34]:
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))


YORK:
You, friends, pardon, my loving speech marks,
Did not anot hear.

STANLEY:
But, my lord; 'tis not thinks.
What's on your hand; 'dness e'er in your great me.

LOVER:
Thrive being out an either broke.

HASTINGS:
Marry, let hear i have these cause on him
Against his business.

MARCIUS:
O farewell the pedlick,
I will pity the gives of Magid firm our self.

MARCIUS:
Well, how shall wear his househind exposters fore
World' Inquirt-reason; therein Coxio; here's Saint.

CORIOLANUS:
I know he did, 
